In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc, plot_precision_recall_curve, roc_curve
from sklearn.metrics import plot_precision_recall_curve, average_precision_score, precision_recall_curve, auc, recall_score, precision_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.preprocessing import MaxAbsScaler
# check version number
from collections import Counter
from imblearn.under_sampling import NearMiss
from sklearn.feature_selection import SelectFromModel, SelectKBest, RFE
from sklearn.decomposition import PCA
import collections


In [20]:
def confuMatrix_measures(y_pred,y_test):
    print('\nConfusion matrix of Random Forest on the test data:')
    confu_mtrx = pd.DataFrame(confusion_matrix(y_test, y_pred, labels=[0,1]), 
                              columns=['pred_neg', 'pred_pos'], index=['neg', 'pos'])    
    print(confu_mtrx)
    
    TP = confu_mtrx.iloc[1,1]
    TN = confu_mtrx.iloc[0,0]
    FP = confu_mtrx.iloc[0,1]
    FN = confu_mtrx.iloc[1,0]
    
    print("TP:", TP)
    print("TN:", TN)
    print("FP:", FP)
    print("FN:", FN)
    
    Sensitivity = TP/(TP+FN) #pred_+ of all + also called recall
    Specifity = TN/(TN + FP) #pred_- of all -
    Precision = TP/(TP+FP) # + of all pred_+
    Accuracy = (TP+TN)/(TP+TN+FP+FN) # right prediction of all prediction
    FPR = FP/(FP+TN) #False_positive_rate, pred_+ but in fact - of all -
    PPV = TP/(TP+FP) #Positive predictive value, real+ in all +_pred
    NPV = TN/(FN+TN) #Negative predictive value, real- in all -pred
    BER = 1/2*(FPR + FN/(FN+TP)) #balanced positive rate
    record_df = pd.DataFrame(
        {'Sensitivity': [Sensitivity],'Specifity':[Specifity], 
         'Precision':[Precision], 'Accuracy':[Accuracy], "FRP":[FPR], "BER":[BER],
         'PPV':[PPV], 'NPV': [NPV], "TP": [TP], "TN": [TN],"FP": [FP],"FN": [FN]}, 
        columns=['Sensitivity', 'Specifity', 'Precision', 'Accuracy', 
                 'PPV', 'NPV', "TP", "TN","FP","FN"]
        )
    return record_df

### Enrichment Factor
def EF(y_prob, y_test, n_percent):
    actives_list = y_test
    score_list = y_prob
    total_actives = len(actives_list[actives_list == 1])
    total_compounds = len(actives_list)
    # Sort scores, while keeping track of active/decoy status
    # NOTE: This will be inefficient for large arrays
    print(zip(score_list, actives_list))
    labeled_hits = sorted(zip(score_list, actives_list), reverse=True)
    # Get top n percent of hits
    num_top = int(total_compounds * n_percent)
    top_hits = labeled_hits[0:num_top]    
    num_actives_top = len([value for score, value in top_hits if value == 1])
    # Calculate enrichment factor
    return num_actives_top / (total_actives * n_percent)

#calculate top hit rate
def tophitrate_per(y_prob, y_test, n_percent):
    pos_al = pd.DataFrame({'prob':y_prob,'Label':y_test})
    num_per = int(len(y_test)*n_percent)
    pos_top = pos_al.nlargest(num_per, 'prob')
    num_active = pos_top['Label'].value_counts()[1]
    return num_active/num_per
#obtain a table of top hit rate and enrichment factor of single model
def EFTH(y_prob, y_test,thresh_list = [0.05,0.02, 0.01, 0.005], M="SO"):
    SB_EF1 =[]
    for n in thresh_list:
        ef_RFscore = EF(y_prob, y_test, n)
        SB_EF1.append(ef_RFscore) 
        #print("RFScore enrichment factor at {}: {:.3f}".format(n, ef_RFscore))

    SB_TH1 =[]
    for n in thresh_list:
        th_RFscore = tophitrate_per(y_prob, y_test, n) 
        SB_TH1.append(th_RFscore) 
        #print("tophit rate at {}: {:.3f}".format(n, th_RFscore)) 
    df = pd.DataFrame({M+'_TH':SB_TH1,M+'_EF':SB_EF1},index=thresh_list)
    df = df.T
    return df
#calculate AUCPR value
def AUCPR_value(y_prob, y_test):
    # calculate the no skill line as the proportion of the positive class
    precision, recall, _ = precision_recall_curve(y_test, y_prob)
    # Use AUC function to calculate the area under the curve of precision recall curve
    auc_precision_recall = auc(recall, precision)
    print('AUCPR:',auc_precision_recall)
    return auc_precision_recall

#plot a PR curve
def PR_curve(y_prob, y_test,atitle=''):
    # calculate the no skill line as the proportion of the positive class
    y = y_test
    no_skill = len(y[y==1]) / len(y)
    # plot the no skill precision-recall curve
    plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
    # calculate model precision-recall curve
    precision, recall, _ = precision_recall_curve(y_test, y_prob)
    # Use AUC function to calculate the area under the curve of precision recall curve
    auc_precision_recall = auc(recall, precision)
    print('AUCPR:',auc_precision_recall)
    # plot the model precision-recall curve
    plt.plot(recall, precision, marker='.', label=f'RF (AUCPR = {round(auc_precision_recall,3)})')
    # axis labels
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # show the legend
    plt.legend()
    plt.title(atitle)
    # show the plot
    #plt.savefig(save_path, dpi=200)
    plt.show()

def Model_Assessment(y_test,y_pred,y_prob,M="SO"):
    print("Model assessment of "+M+" Model")
    print("\n1)Confusion Matrix")
    confuMatrix_measures(y_pred,y_test)
    print("\n2)Table of Top Hit Rate and Enrichment Factor")
    display(EFTH(y_prob, y_test,[0.05,0.02,0.01,0.005,0.001], M))
    print("\n3)Precision Recall Curve and AUCPR")
    PR_curve(y_prob, y_test,atitle="Precision Recall Curve of "+M+" Model")

In [21]:
def con_dp(X_test,y_test,dft1,model):
    yhat = model.predict_proba(X_test)
    pos_probs = yhat[:, 1]
    pos_df = pd.DataFrame({'prob':pos_probs,'Label':y_test,'dorp':dft1['Pose/decoy']})
    #prediction
    positive = pos_df[pos_df['prob'] >0.5]
    negative = pos_df[pos_df['prob']<0.5]
    print('True posive:')
    print(Counter(positive['dorp'][positive['Label']==1]))
    print('False positive:')
    print(Counter(positive['dorp'][positive['Label']==0]))
    print('True neg:')
    print(Counter(negative['dorp'][negative['Label']==0]))
    print('False neg:')
    print(Counter(negative['dorp'][negative['Label']==1]))



# Model Preparation

# Imbalanced test set

In [13]:
dft1 = pd.read_csv("Single-pose_Testset.csv")
dft2 = pd.read_csv("Multi-pose_Testset.csv")


# SB on original test set

In [48]:
dft1_result = dft1.iloc[:,-4:]
dft2_result = dft2.iloc[:,-4:]

In [7]:
'''SB'''
X_train = pd.read_csv('SB_0.csv').drop("Label",1)
y_train = pd.read_csv('SB_0.csv')['Label']

model_SB = RandomForestClassifier(n_estimators=498,min_samples_split=5, min_samples_leaf=1, max_features='sqrt',max_depth=100,bootstrap=True)
model_SB.fit(X_train, y_train)

RandomForestClassifier(max_depth=100, min_samples_split=5, n_estimators=498)

In [16]:
'''SB'''
final_feature_list = X_train.columns.to_list()
len(final_feature_list)
X_test_SB = dft1[final_feature_list]   #Feature Matrix
print('X_test Shape',X_test_SB.shape)
y_test_SB = dft1["Label"]          #Target Variable
print('y_test Shape',y_test_SB.shape)

X_test Shape (5621, 837)
y_test Shape (5621,)


In [50]:
y_prob_SB = model_SB.predict_proba(X_test_SB)[:, 1]
y_pred_SB = model_SB.predict(X_test_SB)
#Model_Assessment(y_test,y_pred,y_prob,"SB")
dft1_result["SB_pred"] = y_pred_SB
dft1_result["SB_prob"] = y_prob_SB


# MB on original multpo set

In [39]:
X_train = pd.read_csv('MB_0.csv').drop("Label",1)
y_train = pd.read_csv('MB_0.csv')['Label']
model_MB = RandomForestClassifier(n_estimators=568,min_samples_split=5, min_samples_leaf=1, max_features='sqrt',max_depth=50,bootstrap=False)
model_MB.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=50, min_samples_split=5,
                       n_estimators=568)

In [41]:
X_train = pd.read_csv('MB_0.csv').drop("Label",1)
y_train = pd.read_csv('MB_0.csv')['Label']
final_feature_list = X_train.columns.to_list()
len(final_feature_list)
X_test_MB = dft2[final_feature_list]   #Feature Matrix
print('X_test Shape',X_test_MB.shape)
y_test_MB = dft2["Label"]          #Target Variable
print('y_test Shape',y_test_MB.shape)
#X_test = X_test_MB.fillna(0)
#y_test = y_test_MB.fillna(0)


X_test Shape (7571, 889)
y_test Shape (7571,)


In [55]:
y_prob_MB = model_MB.predict_proba(X_test_MB)[:, 1]
y_pred_MB = model_MB.predict(X_test_MB)
#Model_Assessment(y_test,y_pred,y_prob,"MB")
dft2_result["MB_pred"] = y_pred_MB
dft2_result["MB_prob"] = y_prob_MB
dft2_result

,PDBCode,Pose/decoy,Pose name,Label,MB_pred,MB_prob
0,2i3i,decoy,2i3i_decoy_9_pose_1,0,0,0.305982
1,4xtv,decoy,4xtv_decoy_5_pose_4,0,0,0.244196
2,3fjg,decoy,3fjg_decoy_5_pose_3,0,0,0.335451
3,2vzr,decoy,2vzr_decoy_6_pose_2,0,0,0.299855
4,5caq,pose,5caq_pose_3,1,1,0.959360
...,...,...,...,...,...,...
7566,1jmg,decoy,1jmg_decoy_2_pose_5,0,0,0.312508
7567,1g7v,decoy,1g7v_decoy_7_pose_5,0,0,0.245859
7568,3f15,decoy,3f15_decoy_1_pose_4,0,0,0.210801
7569,2i4u,decoy,2i4u_decoy_10_pose_2,0,0,0.306148


In [112]:
con_dp(X_test,y_test,dft2,model_MB)

True posive:
Counter({'pose': 618})
False positive:
Counter({'decoy': 287, 'pose': 173})
True neg:
Counter({'decoy': 4809, 'pose': 1428})
False neg:
Counter({'pose': 256})


# so

In [43]:
'''SO'''
X_train = pd.read_csv('SO_0.csv').drop("Label",1)
y_train = pd.read_csv('SO_0.csv')['Label']


model_SO = RandomForestClassifier(n_estimators=568,min_samples_split=5, min_samples_leaf=1, max_features='sqrt',max_depth=50,bootstrap=False)
model_SO.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=50, min_samples_split=5,
                       n_estimators=568)

In [58]:
X_train = pd.read_csv('SO_0.csv').drop("Label",1)
y_train = pd.read_csv('SO_0.csv')['Label']

X_test = pd.read_csv('SOt_0.csv').drop("Label",1)
y_test = pd.read_csv('SOt_0.csv')['Label']

y_prob_SO = model_SO.predict_proba(X_test)[:, 1]
y_pred_SO = model_SO.predict(X_test)

#Model_Assessment(y_test,y_pred,y_prob,"SO")

dft1_result["SO_pred"] = y_pred_SO
dft1_result["SO_prob"] = y_prob_SO
dft1_result

,PDBCode,Pose/decoy,Pose name,Label,SB_pred,SB_prob,SO_pred,SO_prob
0,2i3i,decoy,2i3i_decoy_9_pose_1,0,1,0.500052,0,0.021355
1,4xtv,decoy,4xtv_decoy_5_pose_1,0,1,0.651529,0,0.017549
2,3fjg,decoy,3fjg_decoy_5_pose_1,0,1,0.609230,0,0.006042
3,2vzr,decoy,2vzr_decoy_6_pose_1,0,1,0.694578,0,0.047786
4,5caq,pose,5caq_pose_1,1,1,0.538698,0,0.199935
...,...,...,...,...,...,...,...,...
5616,1jmg,decoy,1jmg_decoy_2_pose_1,0,0,0.440318,0,0.001089
5617,1g7v,decoy,1g7v_decoy_7_pose_1,0,1,0.527584,0,0.010948
5618,3f15,decoy,3f15_decoy_1_pose_1,0,0,0.148085,0,0.061406
5619,2i4u,decoy,2i4u_decoy_10_pose_1,0,1,0.885339,0,0.021554


## MO

In [59]:
'''MO'''
X_train = pd.read_csv('MO_0.csv').drop("Label",1)
y_train = pd.read_csv('MO_0.csv')['Label']

model_MO = RandomForestClassifier(n_estimators=262,min_samples_split=5, min_samples_leaf=1, max_features='sqrt',max_depth=None,bootstrap=True)
model_MO.fit(X_train, y_train)


RandomForestClassifier(min_samples_split=5, n_estimators=262)

In [60]:
'''MO'''
X_train = pd.read_csv('MO_0.csv').drop("Label",1)
y_train = pd.read_csv('MO_0.csv')['Label']
X_test = pd.read_csv('MOt_0.csv').drop("Label",1)
y_test = pd.read_csv('MOt_0.csv')['Label']

y_prob_MO = model_MO.predict_proba(X_test)[:, 1]
y_pred_MO = model_MO.predict(X_test)

#Model_Assessment(y_test,y_pred,y_prob,"MO")
dft2_result["MO_pred"] = y_pred_MO
dft2_result["MO_prob"] = y_prob_MO
dft2_result

,PDBCode,Pose/decoy,Pose name,Label,MB_pred,MB_prob,MO_pred,MO_prob
0,2i3i,decoy,2i3i_decoy_9_pose_1,0,0,0.305982,0,0.046565
1,4xtv,decoy,4xtv_decoy_5_pose_4,0,0,0.244196,0,0.009160
2,3fjg,decoy,3fjg_decoy_5_pose_3,0,0,0.335451,0,0.021947
3,2vzr,decoy,2vzr_decoy_6_pose_2,0,0,0.299855,0,0.023473
4,5caq,pose,5caq_pose_3,1,1,0.959360,1,0.823555
...,...,...,...,...,...,...,...,...
7566,1jmg,decoy,1jmg_decoy_2_pose_5,0,0,0.312508,0,0.000000
7567,1g7v,decoy,1g7v_decoy_7_pose_5,0,0,0.245859,0,0.056820
7568,3f15,decoy,3f15_decoy_1_pose_4,0,0,0.210801,0,0.044911
7569,2i4u,decoy,2i4u_decoy_10_pose_2,0,0,0.306148,0,0.018066
